In [1]:
import pandas as pd

In [2]:
hof = pd.read_parquet('../data/bd/hall_of_fame.parquet')
hof

,player_id,year_id,voted_by,ballots,needed,votes,inducted,category,needed_note
0,cobbty01,1936,BBWAA,226.0,170.0,222.0,Y,Player,None
1,ruthba01,1936,BBWAA,226.0,170.0,215.0,Y,Player,None
2,wagneho01,1936,BBWAA,226.0,170.0,215.0,Y,Player,None
3,mathech01,1936,BBWAA,226.0,170.0,205.0,Y,Player,None
4,johnswa01,1936,BBWAA,226.0,170.0,189.0,Y,Player,None
...,...,...,...,...,...,...,...,...,...
4186,lidgebr01,2018,BBWAA,422.0,317.0,0.0,N,Player,None
4187,millwke01,2018,BBWAA,422.0,317.0,0.0,N,Player,None
4188,zambrca01,2018,BBWAA,422.0,317.0,0.0,N,Player,None
4189,morrija02,2018,Veterans,NaN,NaN,NaN,Y,Player,None


In [3]:
people = pd.read_parquet('../data/bd/people.parquet')[['player_id', 'retro_id', 'name_first', 'name_last']]
people

,player_id,retro_id,name_first,name_last
0,aardsda01,aardd001,David,Aardsma
1,aaronha01,aaroh101,Hank,Aaron
2,aaronto01,aarot101,Tommie,Aaron
3,aasedo01,aased001,Don,Aase
4,abadan01,abada001,Andy,Abad
...,...,...,...,...
19873,zupofr01,zupof101,Frank,Zupo
19874,zuvelpa01,zuvep001,Paul,Zuvella
19875,zuverge01,zuveg101,George,Zuverink
19876,zwilldu01,zwild101,Dutch,Zwilling


In [4]:
hofers = pd.merge(hof[hof['inducted']=='Y'].player_id, people, on='player_id')
hofers

,player_id,retro_id,name_first,name_last
0,cobbty01,cobbt101,Ty,Cobb
1,ruthba01,ruthb101,Babe,Ruth
2,wagneho01,wagnh101,Honus,Wagner
3,mathech01,mathc102,Christy,Mathewson
4,johnswa01,johnw102,Walter,Johnson
...,...,...,...,...
318,guerrvl01,guerv001,Vladimir,Guerrero
319,thomeji01,thomj002,Jim,Thome
320,hoffmtr01,hofft001,Trevor,Hoffman
321,morrija02,morrj001,Jack,Morris


In [5]:
pa_data_cached = True
if pa_data_cached:
    pa = pd.read_parquet('../data/mine/events_hr.parquet')
else:
    gms = pd.read_parquet('../data/mine/gamelog_enhanced.parquet')[['game_id', 'date', 'game_type']]
    ev = pd.read_parquet('../data/retro/event.parquet')[['bat_team_id', 'fld_team_id', 'bat_id', 'pit_id', 'bat_event_fl', 'bat_hand_cd', 'pit_hand_cd', 'event_cd', 'game_id', 'ph_fl', 'bat_lineup_id']]
    pa = ev[(ev['bat_event_fl'])&(ev['event_cd']==23)]
    pa = pd.merge(left=gms, right=pa, on='game_id')
    pa['yr'] = pa['date'].apply (lambda dt: dt.year)
    pa.to_parquet('../data/mine/events_hr.parquet')
pa

,game_id,date,game_type,bat_team_id,fld_team_id,bat_id,pit_id,bat_event_fl,bat_hand_cd,pit_hand_cd,event_cd,ph_fl,bat_lineup_id,yr
0,BOS190310010,1903-10-01,PS,PIT,BOS,sebrj101,younc102,True,L,R,23,False,7,1903
1,BOS190310020,1903-10-02,PS,BOS,PIT,dougp102,leevs101,True,L,R,23,False,1,1903
2,BOS190310020,1903-10-02,PS,BOS,PIT,dougp102,veilb101,True,L,R,23,False,1,1903
3,CHN190810110,1908-10-11,PS,CHN,DET,tinkj101,donob101,True,R,R,23,False,7,1908
4,PIT190910080,1909-10-08,PS,PIT,DET,clarf101,mullg102,True,L,R,23,False,3,1909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292431,HOU201910290,2019-10-29,PS,WAS,HOU,sotoj001,verlj001,True,L,R,23,False,4,2019
292432,HOU201910290,2019-10-29,PS,WAS,HOU,renda001,harrw002,True,R,R,23,False,3,2019
292433,HOU201910300,2019-10-30,PS,HOU,WAS,gurry001,schem001,True,R,R,23,False,5,2019
292434,HOU201910300,2019-10-30,PS,WAS,HOU,renda001,greiz001,True,R,R,23,False,3,2019


In [17]:
hr_off_hofers = pa[pa['pit_id'].isin(hofers.retro_id)].groupby('bat_id')['game_id'].count().sort_values(ascending=False).head(100)
hr_off_hofers

bat_id
aaroh101    73
starw101    62
maysw101    58
mccow101    58
bencj101    52
            ..
pepij101    21
simmt001    21
singk001    21
fairr101    21
dawsa001    21
Name: game_id, Length: 100, dtype: int64

In [13]:
def format_row_reddit(row):
    print(f'|{row.name_first} {row.name_last}|{row.game_id}|')

def format_row_lounge(row):
    name_str = f'{row.name_first} {row.name_last}'
    print(f'{name_str:.<20}{row.game_id}')

In [18]:
pd.merge(hr_off_hofers, people, left_on='bat_id', right_on='retro_id').apply(format_row_lounge, axis=1)
None

Hank Aaron..........73
Willie Stargell.....62
Willie Mays.........58
Willie McCovey......58
Johnny Bench........52
Babe Ruth...........52
Jimmie Foxx.........51
Ernie Banks.........49
Billy Williams......48
Tony Perez..........47
Eddie Mathews.......44
Frank Robinson......43
Joe Torre...........42
Ted Williams........42
Orlando Cepeda......42
Ron Santo...........42
Yogi Berra..........40
Roberto Clemente....39
John Mayberry.......37
Jim Wynn............37
Mickey Mantle.......36
Dick Allen..........36
Stan Musial.........36
Graig Nettles.......36
Al Simmons..........35
Dave Kingman........35
Ken Boyer...........35
Harmon Killebrew....34
Reggie Jackson......34
Willie Davis........34
Lou Gehrig..........34
Joe Morgan..........33
Darrell Evans.......33
Cecil Cooper........33
Duke Snider.........33
Mike Schmidt........32
Rusty Staub.........32
Joe DiMaggio........32
Hank Greenberg......31
Nate Colbert........31
Bobby Bonds.........31
Ron Cey.............31
Charlie Gehringer...31
Richie Hebn